# Pruebas de modelación
Durante este proceso se pondrán a prueba distintos modelos de Machine Learning de entre los cuales se encuentran: Regresión Logistica, Naive Bayes, K-Nearest, Support Vector Machines, Arboles de Decisión y Random Forest. El objetivo de estas pruebas es evaluar que modelo presenta mejores resultados acorde a la problematica abordada para la predicción de la la muerte o supervivencia de el set de datos de el Titanic descrito en reportes anteriores.

### Importando librerias

In [38]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

### Leyendo Set De Datos

In [26]:
df = pd.read_csv('E:/Github/Reto-3006C-equipo5/retro/M4_Reto/Data/train_clean.csv')

### Separación de datos de entrenamiento y prueba

In [27]:
X = df.drop("Survived", axis=1)
y = df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((708, 15), (178, 15))

### Pruebas de modelos

#### Random forest

In [61]:
# Haremos una prueba con un random forest
rf = RandomForestClassifier(random_state=42)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

testing_accuracy = accuracy_score(y_test, y_pred)
training_accuracy = accuracy_score(y_train, rf.predict(X_train))

print('Random Forest: {:.3f}'.format(testing_accuracy))
print('Random Forest: {:.3f}'.format(training_accuracy))

Random Forest: 0.764
Random Forest: 0.986


Al ejecutar un modelo Random Forest en nuestros datos de prueba y entrenamiento, observamos que el rendimiento en los datos de entrenamiento es excepcionalmente alto (aproximadamente 98.6%), mientras que en los datos de prueba es significativamente menor (alrededor del 76.4%). Esta discrepancia entre el rendimiento en entrenamiento y prueba sugiere la presencia de overfitting. En otras palabras, el modelo está aprendiendo demasiado los detalles específicos de los datos de entrenamiento y tiene dificultades para generalizar adecuadamente a nuevos datos. Para abordar este problema, exploraremos el uso de modelos con regularización en un intento de mejorar la capacidad de generalización y obtener resultados más confiables.

In [84]:
clf = RandomForestClassifier()

param_grid = {
    'n_estimators': [50, 100, 200],      
    'max_depth': [None, 10, 20, 30],      
    'min_samples_split': [2, 5, 10],      
    'min_samples_leaf': [1, 2, 4],        
    'bootstrap': [True, False],          
    'criterion': ['gini', 'entropy'] 
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='accuracy',       
    cv=5,                     
    verbose=1,               
    n_jobs=-1                   
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_clf = grid_search.best_estimator_

testing_accuracy = accuracy_score(y_test, best_clf.predict(X_test))
training_accuracy = accuracy_score(y_train, best_clf.predict(X_train))

print('Random Forest: {:.3f}'.format(testing_accuracy))
print('Random Forest: {:.3f}'.format(training_accuracy))

Fitting 5 folds for each of 432 candidates, totalling 2160 fits
{'bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 50}
Random Forest: 0.809
Random Forest: 0.893


Tras aplicar la optimización de parámetros en nuestro modelo Random Forest, observamos una mejora significativa en el rendimiento. Ahora, en los datos de prueba, obtenemos un accuracy del aproximadamente 80.9%, en comparación con el 76.4% obtenido anteriormente sin optimización. En los datos de entrenamiento, el rendimiento también ha mejorado, alcanzando un accuracy de alrededor del 89.3%.

Sin embargo, a pesar de estas mejoras, todavía enfrentamos un desafío importante: el modelo sigue mostrando signos de overfitting. La discrepancia entre el rendimiento en entrenamiento y prueba sugiere que el modelo aún tiene dificultades para generalizar completamente a nuevos datos. 

#### Logistic Regression

In [91]:
# Haremos una prueba con un logistic regression
lr = LogisticRegression(random_state=42)

lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

testing_accuracy = accuracy_score(y_test, y_pred)
training_accuracy = accuracy_score(y_train, lr.predict(X_train))

print('Logistic Regression: {:.3f}'.format(testing_accuracy))
print('Logistic Regression: {:.3f}'.format(training_accuracy))

Logistic Regression: 0.775
Logistic Regression: 0.847


Al aplicar un modelo de regresión logística a nuestros datos, observamos un rendimiento del aproximadamente 84.7% en los datos de entrenamiento, mientras que en los datos de prueba obtenemos un accuracy de alrededor del 77.5%. Esta discrepancia entre el rendimiento en entrenamiento y prueba sugiere una vez más la presencia de overfitting, donde el modelo está aprendiendo demasiado los detalles específicos de los datos de entrenamiento y tiene dificultades para generalizar a nuevos datos.

Dada esta observación, planeamos abordar el overfitting mediante la aplicación de modelos de regresión logística con regularización. 

In [89]:
clf = LogisticRegression(solver='saga')

param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet'],     
    'C': [0.001, 0.01, 0.1, 1, 2, 5, 10, 100],          
    'l1_ratio': [0, 0.1, 0.2, 0.5, 1],
    'max_iter': [500, 1000, 2000, 3000, 5000, 10000]
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='accuracy',         
    cv=5,                       
    verbose=1,                 
    n_jobs=-1                   
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_clf = grid_search.best_estimator_

testing_accuracy = accuracy_score(y_test, best_clf.predict(X_test))
training_accuracy = accuracy_score(y_train, best_clf.predict(X_train))

print('Logistic Regression: {:.3f}'.format(testing_accuracy))
print('Logistic Regression: {:.3f}'.format(training_accuracy))

Fitting 5 folds for each of 720 candidates, totalling 3600 fits
{'C': 2, 'l1_ratio': 0.5, 'max_iter': 5000, 'penalty': 'elasticnet'}
Logistic Regression: 0.775
Logistic Regression: 0.833


Después de aplicar la regularización y optimizar los parámetros en nuestro modelo de regresión logística, observamos mejoras notables en el rendimiento. En los datos de prueba, ahora obtenemos un accuracy del aproximadamente 83.3%, en comparación con el 77.5% obtenido anteriormente sin regularización. En los datos de entrenamiento, el rendimiento también ha mejorado, alcanzando un accuracy de alrededor del 83.3%.

Estos resultados indican que la regularización ha sido efectiva para reducir el overfitting que observamos previamente en el modelo de regresión logística sin regularización

#### XGBoost

In [64]:
# Haremos una prueba con un xgboost
xgb = XGBClassifier(random_state=42)

xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

testing_accuracy = accuracy_score(y_test, y_pred)
training_accuracy = accuracy_score(y_train, xgb.predict(X_train))

print('XGBoost: {:.3f}'.format(testing_accuracy))
print('XGBoost: {:.3f}'.format(training_accuracy))

XGBoost: 0.787
XGBoost: 0.973


Al probar un modelo de XGBoost en nuestros datos, observamos un accuracy del aproximadamente 97.3% en los datos de entrenamiento, mientras que en los datos de prueba obtenemos un accuracy de alrededor del 78.7%. Esta discrepancia significativa en el rendimiento entre entrenamiento y prueba indica una vez más la presencia de overfitting, donde el modelo está aprendiendo demasiado los detalles específicos de los datos de entrenamiento y tiene dificultades para generalizar a nuevos datos.

Dado este desafío, nuestra estrategia para abordar el overfitting es utilizar Grid Search para optimizar los parámetros del modelo XGBoost

In [79]:
clf = XGBClassifier()

param_grid = {
    'alpha': [0.1, 0.2, 0.5, 1, 2, 10],             
    'lambda': [0.05, 0.1, 0.2, 0.5, 1, 2, 10],            
    'gamma': [0, 0.1, 0.5, 1],             
    'max_depth': [1, 2, 3, 4, 5],
    'min_child_weight': [1, 2, 5, 10, 15, 20]   
}

grid_search = GridSearchCV(
    estimator=clf,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,                     
    verbose=1,            
    n_jobs=-1        
)

grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

best_clf = grid_search.best_estimator_

testing_accuracy = accuracy_score(y_test, best_clf.predict(X_test))
training_accuracy = accuracy_score(y_train, best_clf.predict(X_train))

print('XGBoost: {:.3f}'.format(testing_accuracy))
print('XGBoost: {:.3f}'.format(training_accuracy))

Fitting 5 folds for each of 5040 candidates, totalling 25200 fits
{'alpha': 0.5, 'gamma': 0.5, 'lambda': 0.1, 'max_depth': 2, 'min_child_weight': 1}
XGBoost: 0.758
XGBoost: 0.884


Después de aplicar Grid Search para optimizar los parámetros en nuestro modelo XGBoost, observamos una mejora en el rendimiento. En los datos de prueba, ahora obtenemos un accuracy del aproximadamente 88.4%, en comparación con el 78.7% obtenido anteriormente sin la optimización de parámetros. En los datos de entrenamiento, el rendimiento también ha mejorado, alcanzando un accuracy de alrededor del 75.8%.

A pesar de estas mejoras, todavía enfrentamos el desafío del overfitting. La discrepancia entre el rendimiento en entrenamiento y prueba indica que el modelo todavía tiene dificultades para generalizar completamente a nuevos datos

#### Voting Classifier

In [86]:
# Se crea una pipeline para cada modelo

# Random Forest a esta no se le aplica un scaler ya que al ser un modelo de arboles por steps todas las variables se reciben como independientes
rf_pipeline = Pipeline([
    ("rf", RandomForestClassifier(bootstrap=True, criterion='entropy', max_depth=None, min_samples_leaf=2, min_samples_split=10, n_estimators=50, random_state=42))
])

# Logistic Regression estamos utilizando los parametros obtenidos en las pruebas individuales de este modelo
lr_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(penalty='elasticnet', C=1.0, solver='saga', l1_ratio=0.5, max_iter=1000, random_state=42))
])

xgb_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBClassifier(alpha=0.1, gamma=0.5, max_depth=2, min_child_weight=1, random_state=42))
])

# Se crea un Voting Classifier con los modelos anteriores
voting_classifier = VotingClassifier(estimators=[
    ("rf", rf_pipeline),
    ("lr", lr_pipeline),
    ("xgb", xgb_pipeline)
], voting="soft")

# Se entrena el Voting Classifier
voting_classifier.fit(X_train, y_train)

testing_accuracy = accuracy_score(y_test, y_pred)
training_accuracy = accuracy_score(y_train, voting_classifier.predict(X_train))

print('Voting Classifier: {:.3f}'.format(testing_accuracy))
print('Voting Classifier: {:.3f}'.format(training_accuracy))

Voting Classifier: 0.764
Voting Classifier: 0.879


Tras entrenar el Voting Classifier, hemos observado que este modelo presenta un nivel de overfitting superior al modelo de Logistic Regression sin regularización. Sin embargo, al analizar la estabilidad de los resultados, notamos que el modelo de Logistic Regression sin regularización exhibe cambios significativos en el rendimiento entre cada iteración si se permite que sea aleatorio. Esto sugiere una alta variabilidad en los resultados.

En este contexto, hemos tomado la decisión de sacrificar un poco de overfitting en favor de obtener un modelo más estable y consistente. A pesar de que el Voting Classifier muestra un mayor overfitting, su rendimiento es más predecible y constante en comparación con el modelo de Logistic Regression. Esto es esencial, especialmente si planeamos implementar este modelo en un entorno de producción donde la estabilidad y la consistencia son fundamentales.

En resumen, si bien el Voting Classifier tiene un mayor overfitting en comparación con el modelo de Logistic Regression, su estabilidad y consistencia en los resultados son atributos valiosos que respaldan la elección de este último como el modelo preferido para resolver nuestro problema de clasificación binaria.